In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
[time_range, KL_list, etaWC_list, win_jam] = zload(data_folder + 'detec_results.pkz')

In [5]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(715, 715, 715, [0.0, 2.0])

In [6]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [7]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [8]:
sample_path_jam_test_data_sorted['2917']

{u'alpha': 3.0,
 u'length': 254.0,
 u'numPts': 12.0,
 u'speed': 1.025,
 u'startTime': 76500.1784,
 u'uuid': u'8565d7a9-74d1-37aa-bd2e-f327cbbcfc86'}

In [9]:
len(set(potential_ano_list)), set(potential_ano_list)

(295,
 {'1285',
  '1286',
  '1287',
  '1288',
  '1289',
  '1290',
  '1291',
  '1292',
  '1293',
  '1294',
  '1295',
  '1296',
  '1297',
  '1298',
  '1299',
  '1300',
  '1301',
  '1302',
  '1303',
  '1304',
  '1305',
  '1306',
  '1307',
  '1308',
  '1309',
  '1310',
  '1311',
  '1312',
  '1313',
  '2959',
  '2960',
  '2961',
  '2962',
  '2963',
  '2964',
  '2965',
  '2966',
  '2967',
  '2968',
  '2969',
  '2970',
  '2971',
  '2972',
  '2973',
  '2974',
  '2975',
  '2976',
  '2977',
  '2978',
  '2979',
  '2980',
  '2981',
  '2982',
  '2983',
  '2984',
  '2985',
  '2986',
  '2987',
  '2988',
  '2989',
  '2990',
  '2991',
  '2992',
  '2993',
  '2994',
  '2995',
  '2996',
  '2997',
  '2998',
  '2999',
  '3000',
  '3001',
  '3002',
  '3003',
  '3004',
  '3005',
  '3006',
  '3007',
  '3008',
  '3009',
  '3010',
  '3011',
  '3012',
  '3013',
  '3014',
  '3015',
  '3016',
  '3017',
  '3018',
  '3019',
  '3020',
  '3021',
  '3022',
  '3023',
  '3024',
  '3025',
  '3026',
  '3027',
  '3028',
  '3

In [10]:
len(sample_path_jam_test_data_sorted_trans)

3881

In [11]:
# Refine the detection results; if a jam's length is larger than 1 km,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > 1000):
        ano_list.append(key)

In [12]:
len(ano_list), ano_list

(32,
 ['1306',
  '1302',
  '3308',
  '3021',
  '3259',
  '2963',
  '3252',
  '1311',
  '1308',
  '3647',
  '3645',
  '3034',
  '3035',
  '3306',
  '3033',
  '2989',
  '3265',
  '788',
  '3005',
  '3000',
  '752',
  '757',
  '2959',
  '3678',
  '742',
  '3657',
  '3232',
  '3236',
  '3234',
  '771',
  '2979',
  '2964'])

In [13]:
for key in potential_ano_list:
    if (sample_path_jam_test_data_sorted[key]['startTime'] > 69900 and 
        sample_path_jam_test_data_sorted[key]['startTime'] < 69901):
        uuid = sample_path_jam_test_data_sorted[key]['uuid']
        print(points_dict_Mar16[uuid][0][2])

In [14]:
with open(data_folder + '/points_dict_Mar16_comp.json', 'r') as json_file:
    points_dict_Mar16_comp = json.load(json_file)

In [15]:
points_dict_Mar16_comp['50a5d058-2b16-3034-bd41-623f96b2829a']

[[[-71.123535, 42.334441],
  [-71.12364, 42.334329],
  [-71.12384, 42.33408],
  [-71.124185, 42.333598],
  [-71.124659, 42.33294],
  [-71.12478, 42.332789],
  [-71.12535, 42.33198],
  [-71.125498, 42.331798],
  [-71.125729, 42.33141],
  [-71.12573, 42.33141],
  [-71.125835, 42.331218],
  [-71.12595, 42.330951],
  [-71.126026, 42.330675],
  [-71.126057, 42.330385],
  [-71.126085, 42.330063]],
 15]

In [16]:
('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')

('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')

In [17]:
jam1 = [42.359786, -71.128121]

jam2 = [42.363025, -71.13641] 

from geopy.distance import vincenty
vincenty(jam1, jam2).meters

771.8372664016924